In [99]:
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd

In [100]:
pagina_web = requests.get("https://www.cenace.gob.mx/graficademanda.aspx")
print(pagina_web) # 200 respuesta positiva

<Response [200]>


In [101]:
soup = BeautifulSoup(pagina_web.text, "html.parser")

In [102]:
datos = []
clave_region = ["ccbc", "ccbcsur", "cccentral", "ccnoreste", "ccnoroeste", 
                "ccnorte", "ccoccidental", "ccoriental", "ccpeninsular", "sinacional"]

In [103]:
region = "ccbcsur"
datos_region = soup.find("li", class_ = region)
datos_region

<li class="ccbcsur">
<span class="titlex">Baja California Sur</span>
<p>
                                Demanda Neta:
                                <span class="bold" id="ContentPlaceHolder1_demandaBCS">597 MW</span>
</p>
<p>
                                Generación Neta:
                                <span class="bold" id="ContentPlaceHolder1_generacionBCS">610 MW</span>
</p>
<p>
                                Pronóstico Neto:
                                <span class="bold" id="ContentPlaceHolder1_enlaceBCS">591 MW</span>
</p>
<p style="font-size: 11px; margin-top:10px">
<span class="bold" id="ContentPlaceHolder1_datetimeBCS">01/08/2023 11:53:00 p. m.</span>
</p>
</li>

In [104]:
for clave in clave_region:
    print(clave)

ccbc
ccbcsur
cccentral
ccnoreste
ccnoroeste
ccnorte
ccoccidental
ccoriental
ccpeninsular
sinacional


In [105]:
fecha = datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")
for clave in clave_region:
        
      titulo_region = soup.find("li", class_ = clave)
      region = titulo_region.select_one(".titlex").text

      p_tags = soup.select("li." + clave + " p") # Selecciono las 3 etiquetas p
      for p_tag in p_tags:
            if "Demanda Neta:" in p_tag.text:
                 demanda_neta = float(p_tag.select_one(".bold").text.strip().replace(",","")[:-3]) 
            elif "Generación Neta" in p_tag.text:
                 generacion_neta = float(p_tag.select_one(".bold").text.strip().replace(",","")[:-3])
            elif "Pronóstico Neto" in p_tag.text:
                 pronostico_neto = p_tag.select_one(".bold").text.strip().replace(",","")[:-3]     
               
      datos.append({
              "Fecha" : fecha,
              "Región": region,
              "Generación Neta MW" : generacion_neta,
              "Demanda Neta MW" : demanda_neta,
              "Diferencia MW" : generacion_neta - demanda_neta,
              "Pronóstico Neto MW" : pronostico_neto
            })
      
df = pd.DataFrame(datos)
df

,Fecha,Región,Generación Neta MW,Demanda Neta MW,Diferencia MW,Pronóstico Neto MW
0,01-08-2023 23:54:47,Baja California,2506.0,2516.0,-10.0,2951
1,01-08-2023 23:54:47,Baja California Sur,610.0,597.0,13.0,591
2,01-08-2023 23:54:47,CENTRAL,4191.0,6228.0,-2037.0,6988
3,01-08-2023 23:54:47,NORESTE,12542.0,10014.0,2528.0,9811
4,01-08-2023 23:54:47,NOROESTE,4771.0,4884.0,-113.0,5056
5,01-08-2023 23:54:47,NORTE,4063.0,4525.0,-462.0,4929
6,01-08-2023 23:54:47,OCCIDENTAL,6264.0,9091.0,-2827.0,9411
7,01-08-2023 23:54:47,ORIENTAL,10749.0,7043.0,3706.0,7521
8,01-08-2023 23:54:47,PENINSULAR,1358.0,2309.0,-951.0,2536
9,01-08-2023 23:54:47,S.I. NACIONAL,43933.0,44060.0,-127.0,00
